In [11]:
import math
import numpy as np

In [13]:
# Funções úteis
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoidArray(array):
    return 1 / (1 + np.exp(-array))

def lostFunction(realValue, calculatedValue):
    return -((realValue * math.log10(calculatedValue)) + ((1-realValue) * math.log10(1-calculatedValue)))

def lostFunctionArray(referenceVector, calculatedVector):
    return -(np.dot(referenceVector, np.log10(calculatedVector).T) + np.dot((1-referenceVector), np.log10(1-calculatedVector).T))

In [14]:
# Variáveis
m = int(4e2)                                            # Quantidade total de dados
nx = int(2e2)                                           # Número de features por entrada

# Geração aleatória dos dados para a simulação da aplicação da regressão logística com descida de gradiente
X = np.random.rand(nx, m)                         # Entrada, podem ser imagens, sons, etc. Cada coluna representa 1 conjunto de dados de entrada.     ##Formato: (nx, m)
Y = np.random.randint(0, 2, m).reshape(1,m)       # Saída(Entrada), pode ser uma classificação de animais, estilo musical, etc.                       ##Formato: (1, m)
TrainingSet = np.array([X, Y]).reshape(2,1)       # Conjunto de todos os pares ordenados (entrada, saída) dos exemplos de treinamento.                ##Formato: (2, 1)

# Inicialização dos parâmetros de ajuste da IA, parâmetros que sofrerão mudanças para tentar prever o resultado da saída
w = np.zeros((nx, 1))                             # Parâmetros de ajuste dos dados de treinamento (iniciado em 0)                                     ##Formato: (nx,1)
b = np.zeros((1,1))                               # Parâmetro de ajuste dos dados de treinamento (iniciado em 0)                                      ##Formato: (1,1)

print("Novos dados carregados com sucesso!\n")

Novos dados carregados com sucesso!



C:\Users\rapha\AppData\Local\Temp/ipykernel_14580/3325167118.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  TrainingSet = np.array([X, Y]).reshape(2,1)       # Conjunto de todos os pares ordenados (entrada, saída) dos exemplos de treinamento.                ##Formato: (2, 1)


In [15]:
print(f"TrainingSet (conjunto de dados), formato {TrainingSet.shape}:\n{TrainingSet}"
      f"\n\nX (Entrada), formato {X.shape}:\n{X}"

      f"\n\nY (Saída), formato {Y.shape}:\n{Y}"
      f"\n\nw (peso por entrada), formato {w.shape}:\n{w}"
      f"\n\nb (peso geral), formato {b.shape}:\n{b}")

TrainingSet (conjunto de dados), formato (2, 1):
[[array([[0.47382248, 0.62274892, 0.50987997, ..., 0.04231031, 0.99493997,
          0.87941767],
         [0.21889921, 0.62624716, 0.07080607, ..., 0.08023248, 0.95420818,
          0.62398077],
         [0.78460576, 0.56331153, 0.73656221, ..., 0.62226172, 0.23887389,
          0.50372281],
         ...,
         [0.54701824, 0.10520484, 0.18248875, ..., 0.36222638, 0.93478279,
          0.29627326],
         [0.8397117 , 0.83635714, 0.90664147, ..., 0.44754792, 0.53292403,
          0.2256433 ],
         [0.43579684, 0.99092865, 0.56526454, ..., 0.33949684, 0.8639327 ,
          0.09012665]])                                                   ]
 [array([[1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
          0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
          0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
          1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1

In [16]:
def accuracyCalc():
    """
    Testa a regressão logística atual comparando os resultados obtidos com os corretos.

    :returns: array = [Valores predizidos - formato(1, m), precisão das predições]
    """
    prediction = np.zeros(m).reshape(1, m)
    rights = 0

    # Preenche o vetor de predição e calcula quantas vezes houve sucesso em prever o valor correto
    Z = np.dot(w.T, X) + b
    A = sigmoidArray(Z)
    prediction = A

    for i in range(m):
        if (prediction[0][i] > 0.5) and (Y[0][i] > 0.5):
            rights += 1
        if (prediction[0][i] <= 0.5) and (Y[0][i] <= 0.5):
            rights += 1

    return [prediction, rights / m]

In [6]:
def gradientDescentStep(input, output, weightVector, bValue, learningRate = 0.5):
    # Funcional para parâmetros com formatos: x = (nx, m), y = (1, m), w = (nx, 1), b = (1, 1)
    J = 0; db = 0; m = output.size; nx = weightVector.size; dw = np.zeros((nx, 1))

    for i in range(m): # Calcular para cada amostra (i) do dataset que contém m amostras
        # Variáveis úteis para cada amostra (i)
        xi = input[:,i]                             # Entrada da amostra (i)
        yi = output[0][i]                           # Saída da amostra (i)

        # Cálculo da estimativa realizada pela regressão logística
        zi = np.dot(weightVector.T, xi) + bValue    # z calculado para a amostra (i)
        ai = sigmoid(zi)                            # Saída estimada, ou a da amostra (i)

        # Calculo do erro da amostra (i) é adicionado ao contador para gerar a função de custo (o quão bem w e b estão ajustados com base em TODAS as amostras)
        J += lostFunction(yi, ai)

        # Cálculo das derivadas
        dz = ai - yi
        for d in range(nx):                         # Para cada um dos parâmetros w, calcula-se a derivada dele e adiciona-se ao contador
            dw[d] += xi[d] * dz                     # xi*dz é o valor de dw(1, 2, ..., nx) da amostra (i)
        db += dz                                    # dz é o valor de db da amostra (i)

    # Cálculo dos valores totais (média de todas as amostras)

    J  /= m                                         # Calculo da função de custo, o quão bem w e b estão ajustados para TODAS as amostras
    dw /= m                                         # Calculo de cada uma das derivadas de w, para se dar um passo em direção ao mínimo global (melhor ajuste)
    db /= m                                         # Calculo da derivada do parâmetro b, para se dar um passo em direção ao mínimo global (melhor ajuste)

    # Passo do gradiente descendente para ajustar os parâmetros da IA em direção ao mínimo global
    weightVector -= learningRate * dw
    bValue -= learningRate * db

    return J

In [35]:
def gradientDescentStepVectorized(input, output, weightVector, bValue, learningRate = 0.5):
    # Funcional para parâmetros com formatos: x = (nx, m), y = (1, m), w = (nx, 1), b = (1, 1)
    m = output.size
    Z = np.dot(weightVector.T, input) + bValue
    A = sigmoidArray(Z)
    J = np.sum(lostFunctionArray(output, A))/m
    dZ = A - output
    dW = np.dot(input, dZ.T) / m
    dB = np.sum(dZ) / m

    weightVector -= learningRate * dW
    bValue -= learningRate * dB

    return J

In [24]:
import time

steps = int(6e4)
learningRate = 0.6

aux = int(steps/30)
tic = time.time_ns()
for i in range(steps):
    gradientDescentStepVectorized(X, Y, w, b, learningRate)
    if i % aux == 0:
        print(f"{i/steps*100:.0f}%\t- Erro cometido:\t{gradientDescentStepVectorized(X, Y, w, b, learningRate)*100:.3f}%"
              f"\t\t- Precisão medida:\t{accuracyCalc()[1]*100:.1f}%")
    if i > int(steps/2):
        learningRate = 0.25
    if i > int(steps/1.3):
        learningRate = 0.05

toc = time.time_ns()

print(f"\nTempo de execução: {(toc-tic)/1e9:.3f} s")

C:\Users\rapha\AppData\Local\Temp/ipykernel_14580/3418056945.py:12: RuntimeWarning: divide by zero encountered in log10
  return -(np.dot(referenceVector, np.log10(calculatedVector).T) + np.dot((1-referenceVector), np.log10(1-calculatedVector).T))


0%	- Erro cometido:	nan%		- Precisão medida:	92.2%
3%	- Erro cometido:	nan%		- Precisão medida:	84.5%
7%	- Erro cometido:	nan%		- Precisão medida:	86.2%
10%	- Erro cometido:	nan%		- Precisão medida:	84.8%
13%	- Erro cometido:	nan%		- Precisão medida:	86.8%
17%	- Erro cometido:	nan%		- Precisão medida:	85.0%
20%	- Erro cometido:	nan%		- Precisão medida:	87.2%
23%	- Erro cometido:	nan%		- Precisão medida:	85.2%
27%	- Erro cometido:	nan%		- Precisão medida:	87.5%
30%	- Erro cometido:	nan%		- Precisão medida:	85.2%
33%	- Erro cometido:	nan%		- Precisão medida:	87.5%
37%	- Erro cometido:	nan%		- Precisão medida:	85.5%
40%	- Erro cometido:	nan%		- Precisão medida:	87.5%
43%	- Erro cometido:	nan%		- Precisão medida:	85.8%
47%	- Erro cometido:	nan%		- Precisão medida:	87.5%
50%	- Erro cometido:	nan%		- Precisão medida:	85.8%
53%	- Erro cometido:	nan%		- Precisão medida:	92.5%
57%	- Erro cometido:	nan%		- Precisão medida:	92.5%
60%	- Erro cometido:	nan%		- Precisão medida:	92.5%
63%	- Erro come

In [9]:
print(f"Precisão da IA: {accuracyCalc()[1]*100:.1f}%\n")
print(f"w:\t\t\tb: {b}\n{w}")

Precisão da IA: 74.8%

w:			b: [[9.98299781]]
[[ 0.2135721 ]
 [-0.04439243]
 [-0.2583364 ]
 [-0.81551103]
 [-0.39861721]
 [-0.0117948 ]
 [ 0.14588146]
 [-0.24795919]
 [-1.53075368]
 [-1.00772658]
 [ 0.3555162 ]
 [-0.68382907]
 [-0.80410044]
 [ 0.20351784]
 [ 0.31936897]
 [-0.33535663]
 [-0.81209401]
 [-0.01948027]
 [-0.58661462]
 [-0.22823052]
 [-1.23506063]
 [ 0.0182734 ]
 [ 0.27208205]
 [-0.37725167]
 [-0.04013558]
 [ 0.61382127]
 [ 0.78103862]
 [-0.26718275]
 [-0.6280734 ]
 [ 0.42655452]
 [-0.53282597]
 [-0.11777138]
 [-0.32554949]
 [ 0.91733465]
 [-0.01733892]
 [ 0.07290484]
 [-0.64122185]
 [-0.00204296]
 [-0.48999312]
 [-0.07716762]
 [-0.73087102]
 [ 0.54605149]
 [-0.95776002]
 [-0.32200714]
 [-0.04003417]
 [-0.45998089]
 [-0.5531308 ]
 [-1.6036961 ]
 [ 1.21396864]
 [ 0.12113172]
 [ 0.17295907]
 [-0.12180049]
 [ 0.34251184]
 [ 0.59459549]
 [-0.07888498]
 [ 0.33134904]
 [ 0.32320074]
 [-0.60376306]
 [-0.02490157]
 [-0.59225256]
 [-0.12030459]
 [-0.46571043]
 [-0.00683861]
 [-0.7170

In [11]:
print(f"Precisão da IA: {accuracyCalc()[1]*100:.1f}%\n")
print(f"w:\t\t\tb: {b}\n{w}")

Precisão da IA: 75.0%

w:			b: [[5.83849371]]
[[ 0.77194976]
 [ 0.1506866 ]
 [-1.09976433]
 [ 0.05835352]
 [-0.69477483]
 [-0.11400431]
 [-0.94244477]
 [ 0.11736175]
 [-0.70913038]
 [ 0.19773582]
 [-1.57835937]
 [ 0.46527052]
 [-1.31945808]
 [ 0.51577631]
 [-0.1198114 ]
 [-0.77661383]
 [-0.39518645]
 [-0.98007789]
 [-0.36687732]
 [-0.20311653]
 [-0.41302628]
 [ 0.15404697]
 [ 0.84526432]
 [ 0.1842165 ]
 [ 0.29155583]
 [-0.85238826]
 [ 0.11176619]
 [-0.00768103]
 [ 0.22992176]
 [ 1.11646298]
 [-0.58273188]
 [-0.00983843]
 [-0.18605715]
 [-0.53426968]
 [-0.17952407]
 [-0.96305473]
 [ 0.77256803]
 [-0.77573155]
 [-0.47045027]
 [ 0.13783302]
 [ 0.76762855]
 [ 0.37996003]
 [-1.04663539]
 [-0.16643575]
 [ 0.15913886]
 [ 0.23289993]
 [-0.16408245]
 [-1.12222097]
 [-0.28733666]
 [-0.5239992 ]
 [-0.34752518]
 [ 0.40779995]
 [ 0.19298278]
 [-0.85564686]
 [ 0.8256765 ]
 [-0.4382954 ]
 [-0.24044299]
 [ 0.36987032]
 [ 0.16079707]
 [ 1.32865377]
 [ 0.5831952 ]
 [ 1.0913743 ]
 [-0.08884485]
 [ 0.4450

In [13]:
import time

stepNumber = 200

# Versão não vetorizada
tic = time.time_ns()
for i in range(stepNumber):
    gradientDescentStep(X, Y, w, b, 0.01)
toc = time.time_ns()
normalGradientTime = (toc - tic)/1e6 # tempo em milissegundos

# Versão vetorizada
tic = time.time_ns()
for i in range(stepNumber):
    gradientDescentStepVectorized(X, Y, w, b, 0.01)
toc = time.time_ns()
vectorizedGradientTime = (toc - tic)/1e6 # tempo em milissegundos

print (f"\n\nTempo de execução não vetorizado    : {normalGradientTime:.5f} ms\n"
       f"Tempo de execução codigo vetorizado : {vectorizedGradientTime:.5f} ms\n\n"
       f"O código vetorizado é {normalGradientTime/vectorizedGradientTime:.1f} vezes mais rápido em relação ao não vetorizado")



Tempo de execução não vetorizado    : 45209.99900 ms
Tempo de execução codigo vetorizado : 48.00030 ms

O código vetorizado é 941.9 vezes mais rápido em relação ao não vetorizado


In [25]:
print(f"Precisão da IA: {accuracyCalc()[1]*100:.2f}%\n")

print(f"\nComparação de valores:"
      f"\n\ncalc\t|\treal")
prediction = accuracyCalc()[0]
for i in range(m):
    print(f"\n{prediction[0][i]:.2f}\t|\t{Y[0][i]:.2f}\t", end="")
    printed = False
    if (prediction[0][i] > 0.5) and (Y[0][i] > 0.5) and not printed:
        print("(Ok)")
        printed = True
    if (prediction[0][i] <= 0.5) and (Y[0][i] <= 0.5) and not printed:
        print("(Ok)")
        printed = True
    if not printed:
        print("(X)")


Precisão da IA: 92.50%


Comparação de valores:

calc	|	real

1.00	|	1.00	(Ok)

0.09	|	1.00	(X)

0.96	|	1.00	(Ok)

1.00	|	1.00	(Ok)

0.00	|	0.00	(Ok)

1.00	|	1.00	(Ok)

0.11	|	0.00	(Ok)

0.20	|	0.00	(Ok)

0.66	|	1.00	(Ok)

0.00	|	0.00	(Ok)

0.15	|	0.00	(Ok)

0.92	|	1.00	(Ok)

0.29	|	0.00	(Ok)

0.17	|	0.00	(Ok)

0.97	|	1.00	(Ok)

0.06	|	0.00	(Ok)

0.04	|	0.00	(Ok)

0.50	|	0.00	(X)

0.00	|	0.00	(Ok)

0.78	|	1.00	(Ok)

0.24	|	0.00	(Ok)

0.27	|	0.00	(Ok)

0.36	|	0.00	(Ok)

0.38	|	0.00	(Ok)

0.86	|	1.00	(Ok)

0.00	|	0.00	(Ok)

0.44	|	1.00	(X)

0.00	|	0.00	(Ok)

0.92	|	1.00	(Ok)

0.11	|	0.00	(Ok)

0.65	|	1.00	(Ok)

1.00	|	1.00	(Ok)

0.08	|	0.00	(Ok)

1.00	|	1.00	(Ok)

1.00	|	1.00	(Ok)

0.00	|	0.00	(Ok)

0.00	|	0.00	(Ok)

0.44	|	0.00	(Ok)

0.00	|	0.00	(Ok)

0.01	|	0.00	(Ok)

0.84	|	1.00	(Ok)

0.00	|	0.00	(Ok)

0.00	|	0.00	(Ok)

1.00	|	1.00	(Ok)

0.00	|	0.00	(Ok)

0.73	|	1.00	(Ok)

0.18	|	0.00	(Ok)

0.75	|	1.00	(Ok)

1.00	|	1.00	(Ok)

0.20	|	0.00	(Ok)

0.00	|	0.00	(Ok)

0.00	|	0.00	(Ok)

1.00	